In [1]:
import osmnx as ox
import geopandas as gpd
import folium
import pandas as pd
from pyproj import CRS

In [43]:
def amenity_score(elements: dict[dict[str]], neigh_file='Neighbourhoods/Neighbourhoods.shp',
                  place='City of Toronto, Golden Horseshoe, Ontario, Canada'):
    
    # Reading in our neighbourhood shapefile 

    neigh = gpd.read_file(neigh_file)

    # Dictionary for storing tags from the OpenStreetMap database
    stored_selections = {}

    # Looping over our elements dictionary and the sets located within
    for ele, val in elements.items():
        for ele2, val2 in val.items():
            selection = ox.geometries_from_place(place, {ele2:val2})
            stored_selections[val2] = selection

    # Creating our final dataframe which will serve to hold neighbourhood names, geometry, area data, and density data        
    final_neigh_joined = gpd.GeoDataFrame(columns = ['neighbourhoods', 'geometry', 'area'])
    final_neigh_joined['geometry'] = neigh['geometry']
    final_neigh_joined['neighbourhoods'] = neigh['FIELD_7']
    final_neigh_joined['area'] = final_neigh_joined['geometry'].area

    # Creating a dictionary to store the dataframes of each individual tag
    maps_list = {}

    for key, val in stored_selections.items():

        # Spatial join joining OSM tags to the Toronto neighbourhood shapefile
        neigh_joined  = gpd.sjoin(neigh, val, how='left', op='intersects')

        # Counting the number of each building/tag that falls into each neighbourhood boundary
        neigh_joined = neigh_joined.groupby('FIELD_7').count()

        # Resetting each dataframe's index, as neighbourhood names are currently acting as its primary index.
        neigh_joined.reset_index(level=0, inplace=True)

        # Selecting our most relevant columns and dropping the rest. FIELD_7 contians neighbourhood names, FIELD_1 contains counts
        neigh_joined = neigh_joined[['FIELD_7', 'FIELD_1']]

        # Renaming the aformentioned columns for clarity
        neigh_joined.rename(columns={'FIELD_1': 'counts'}, inplace=True)
        neigh_joined.rename(columns={'FIELD_7': 'neighbourhoods'}, inplace=True)

        # Calculating the density of tags in each Toronto neighbourhood
        neigh_joined['density'] = neigh_joined['counts']/final_neigh_joined['area']

        # Converting from m2 to km2
        neigh_joined['density'] = neigh_joined['density'].apply(lambda x: x//1000)
        maps_list[key] = neigh_joined

        # Adding each tag's dataframe and their density + count columns to the final dataset
        for key2, val2 in maps_list.items():
            final_neigh_joined[key2] = val2['density']
            final_neigh_joined[key2 + '_counts'] = val2['counts']

    # Saving the columns and alias names of our individual tag names for our final folium display
    keys_folium = list(maps_list.keys())
    col_list = []
    for name in maps_list.keys():
        name = name.capitalize() + ' Density:'
        name = name.replace('_', ' ')
        col_list.append(name)

    # Counts for final density calculations
    keys_counts = []
    for col in final_neigh_joined.keys():
        if '_counts' in col:
            keys_counts.append(col)

    # Final density calculations
    final_neigh_joined['final_count'] = final_neigh_joined.apply(lambda row: row[keys_counts].sum(), axis=1)
    final_neigh_joined['density_total'] = final_neigh_joined['final_count']/final_neigh_joined['area']

    # Converting from m2 to km2
    final_neigh_joined['density_total'] = final_neigh_joined['density_total'].apply(lambda x: x//1000)

    # Creating the 'geoid' field, which is necessary for Folium, and a json file for our Tooltips
    final_neigh_joined['geoid'] = final_neigh_joined.index.astype(str)
    neighjson = final_neigh_joined.to_json()

    # ----- Folium Display -----

    bins = list(final_neigh_joined['density_total'].quantile([0, 0.25, 0.5, 0.75, 1]))

    map_ = folium.Map(location=[43.65, -79.38], tiles = 'cartodbpositron', zoom_start=10, control_scale=True)

    choropleth = folium.Choropleth(
        geo_data=neighjson,
        name='Amenity Densities for the City of Toronto',
        data=final_neigh_joined,
        columns=['geoid', 'density_total'],
        key_on='feature.id',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.4,
        line_color='black', 
        line_weight=0,
        highlight=False, 
        smooth_factor=1.0,
        legend_name= 'Density per Sq.km',
        bins=bins
    ).add_to(map_)

    choropleth.geojson.add_child(
        folium.features.GeoJsonTooltip(['neighbourhoods', 'density_total', *keys_folium],
                                       aliases=['Neighbourhood:', 'Total Amenity Density:', *col_list])
    )
    
    return map_